## Shor's Algorithm

Shor's algorithm is used to find prime factors of an integer. On a quantum computer, Shor's algorithm runs in polynomial time and is almost exponentially faster than the most efficient known classical factoring algorithm. 
The efficiency of Shor's algorithm is due to the efficiency of the Quantum Fourier transform, Quantum Phase estimation and modular exponentiation by repeated squarings. 

In this notebook, you implement the Shor's algorithm in code using the Amazon Braket SDK.

Where:
- n is the integer to be factored
- a is any integer that satisfies 1 < a < n and gcd(a, n) = 1.

## Preparation
We import the functions and modules required to perform the emulation.
A simple function is added to randomize n.

## General steps

* Copy the code into `test.py`, in this case `test.py` is empty and there is no need to run the script from a particular directory.
* Any references/imports are of modules that have been installed prior to the start of this lab
* Run the program: `python test.py` from a terminal to see the results for every step

In [1]:
import random
from math import gcd
from braket.devices import LocalSimulator
from braket.aws import AwsDevice
from braket.experimental.algorithms.shors.shors import (
    shors_algorithm,
    run_shors_algorithm,
    get_factors_from_results
)

def _rand_n(bits: int):
    # bits to decimal
    min_ = pow(2, bits-1)
    max_ = pow(2, bits) - 1
    result = random.randint(min_, max_)
    print("N is set to:", result)
    return result

def main():
    # Main function

    #  #  # Place the other code here #  #  #

    raise SystemExit()


if __name__ == '__main__':
    main()


## Prepare inputs for Shor's Algorithm

### Calculating a
As we need to satisfy `1 < a < n and gcd(a, n) = 1` we add a small loop. Using quantum compute will not bring any significant benefit to this calculation and hence it is performed on the regular CPU.

### Quantum computer's circuit
To perform a computation of Shor's algorithm a *Quantum Circuit* is required. A *Quantum Circuit* is a model for quantum computation, similar to classical circuits, in which a computation is a sequence of quantum gates, measurements, initializations of qubits to known values, and possibly other actions.

We create the circuit by calling the function `shors_algorithm` and share n and a with that function.


In [2]:
def main():

    # ...

    nr_of_bits = 6

    n = _rand_n(nr_of_bits)

    a = 2
    result = []
    while 1 < a < n:
        if gcd(a, n) == 1:
          result.append(a)
        a += 1
    a = random.choice(result)

    shors_circuit = shors_algorithm(n, a)

## Run on a local simulator

Now that we have a circuit we can run a local Quantum simulator or do we?

In [ ]:

def main():

    # ...

    local_simulator = LocalSimulator()

    output = run_shors_algorithm(shors_circuit, local_simulator)

    guessed_factors = get_factors_from_results(output, n, a)

    print(guessed_factors)

## Building the circuit
The SDK has a function `modular_exponentiation_amod15` that builds the actual circuit for performing the calculation.

It clearly has its limitations as shown below.

In [ ]:
def modular_exponentiation_amod15(
    counting_qubits: QubitSetInput, aux_qubits: QubitSetInput, integer_a: int
) -> Circuit:
    """
    Construct a circuit object corresponding the modular exponentiation of a^x Mod 15

    Args:
        counting_qubits (QubitSetInput): Qubits defining the counting register
        aux_qubits (QubitSetInput) : Qubits defining the auxilary register
        integer_a (int) : Any integer that satisfies 1 < a < N and gcd(a, N) = 1.
    Returns:
        Circuit: Circuit object that implements the modular exponentiation of a^x Mod 15
    """

    # Instantiate circuit object
    mod_exp_amod15 = Circuit()

    for x in counting_qubits:
        r = 2**x
        if integer_a not in [2, 7, 8, 11, 13]:
            raise ValueError("integer 'a' must be 2,7,8,11 or 13")
        for iteration in range(r):
            if integer_a in [2, 13]:
                mod_exp_amod15.cswap(x, aux_qubits[0], aux_qubits[1])
                mod_exp_amod15.cswap(x, aux_qubits[1], aux_qubits[2])
                mod_exp_amod15.cswap(x, aux_qubits[2], aux_qubits[3])
            if integer_a in [7, 8]:
                mod_exp_amod15.cswap(x, aux_qubits[2], aux_qubits[3])
                mod_exp_amod15.cswap(x, aux_qubits[1], aux_qubits[2])
                mod_exp_amod15.cswap(x, aux_qubits[0], aux_qubits[1])
            if integer_a == 11:
                mod_exp_amod15.cswap(x, aux_qubits[1], aux_qubits[3])
                mod_exp_amod15.cswap(x, aux_qubits[0], aux_qubits[2])
            if integer_a in [7, 11, 13]:
                for q in aux_qubits:
                    mod_exp_amod15.cnot(x, q)

    return mod_exp_amod15

## Limits of the Circuit

It seems that it is not just the number of qubits or gates but also that the correct circuit is required to perform the calculation.
Creating such a circuit, whether it be for a simulator or for an actual Quantum computer is highly complex and in this case not automated.